In [14]:
# importando libs
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import pickle
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json
from prophet import plot
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [15]:
DAYS_PREDICT = 15

In [16]:
date_today = datetime.today().strftime("%Y-%m-%d")
data_init = "2014-01-01"

df_doge = yf.download("DOGE-USD", data_init, date_today)

df_doge.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-11-09,0.001207,0.001415,0.001181,0.001415,0.001415,6259550
2017-11-10,0.001421,0.001431,0.001125,0.001163,0.001163,4246520
2017-11-11,0.001146,0.001257,0.001141,0.001201,0.001201,2231080
2017-11-12,0.001189,0.001210,0.001002,0.001038,0.001038,3288960
2017-11-13,0.001046,0.001212,0.001019,0.001211,0.001211,2481270


In [17]:
df_doge.reset_index(inplace=True)
df_doge.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
1768,2022-09-12,0.063715,0.065194,0.062878,0.063905,0.063905,383630739
1769,2022-09-13,0.063903,0.063981,0.059524,0.060129,0.060129,461644993
1770,2022-09-14,0.059717,0.061425,0.059542,0.061173,0.061173,369554274
1771,2022-09-15,0.061168,0.061211,0.058785,0.058791,0.058791,327176374
1772,2022-09-16,0.058792,0.060423,0.058453,0.060423,0.060423,247011315


In [18]:
df = df_doge[["Date", "Adj Close"]]
df.rename(columns= { "Date": "ds", "Adj Close": "y" }, inplace=True)

In [19]:
df.tail()

,ds,y
1768,2022-09-12,0.063905
1769,2022-09-13,0.060129
1770,2022-09-14,0.061173
1771,2022-09-15,0.058791
1772,2022-09-16,0.060423


In [20]:
assert df.isnull().sum().sum() == 0
assert df['ds'].isnull().sum() == 0
assert df['y'].isnull().sum() == 0

In [21]:
df.dtypes

ds    datetime64[ns]
y            float64
dtype: object

In [22]:
model = Prophet(seasonality_mode="multiplicative")
model.fit(df)

00:33:56 - cmdstanpy - INFO - Chain [1] start processing
00:33:56 - cmdstanpy - INFO - Chain [1] done processing


In [23]:
df_future = model.make_future_dataframe(periods=DAYS_PREDICT)
df_future.tail()

,ds
1783,2022-09-27
1784,2022-09-28
1785,2022-09-29
1786,2022-09-30
1787,2022-10-01


In [24]:
# predict 
predict = model.predict(df_future)
predict

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2017-11-09,0.004766,-0.021214,0.034161,0.004766,0.004766,0.246540,0.246540,0.246540,-0.004588,-0.004588,-0.004588,0.251128,0.251128,0.251128,0.0,0.0,0.0,0.005941
1,2017-11-10,0.004762,-0.025176,0.032696,0.004762,0.004762,0.257643,0.257643,0.257643,0.014243,0.014243,0.014243,0.243400,0.243400,0.243400,0.0,0.0,0.0,0.005989
2,2017-11-11,0.004759,-0.022493,0.033391,0.004759,0.004759,0.241031,0.241031,0.241031,0.006660,0.006660,0.006660,0.234370,0.234370,0.234370,0.0,0.0,0.0,0.005906
3,2017-11-12,0.004755,-0.024425,0.032840,0.004755,0.004755,0.230319,0.230319,0.230319,0.006155,0.006155,0.006155,0.224164,0.224164,0.224164,0.0,0.0,0.0,0.005850
4,2017-11-13,0.004751,-0.020843,0.034530,0.004751,0.004751,0.204492,0.204492,0.204492,-0.008425,-0.008425,-0.008425,0.212916,0.212916,0.212916,0.0,0.0,0.0,0.005723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,2022-09-27,0.017901,-0.010694,0.045158,0.017901,0.017901,-0.082993,-0.082993,-0.082993,-0.008669,-0.008669,-0.008669,-0.074324,-0.074324,-0.074324,0.0,0.0,0.0,0.016415
1784,2022-09-28,0.017301,-0.011489,0.043311,0.017301,0.017301,-0.076490,-0.076490,-0.076490,-0.005375,-0.005375,-0.005375,-0.071115,-0.071115,-0.071115,0.0,0.0,0.0,0.015977
1785,2022-09-29,0.016701,-0.013321,0.042562,0.016701,0.016701,-0.072095,-0.072095,-0.072095,-0.004588,-0.004588,-0.004588,-0.067507,-0.067507,-0.067507,0.0,0.0,0.0,0.015497
1786,2022-09-30,0.016101,-0.012425,0.041572,0.016101,0.016101,-0.049308,-0.049308,-0.049308,0.014243,0.014243,0.014243,-0.063551,-0.063551,-0.063551,0.0,0.0,0.0,0.015307


In [25]:
predict[["ds", "yhat_lower", "yhat_upper", "yhat"]].tail()

,ds,yhat_lower,yhat_upper,yhat
1783,2022-09-27,-0.010694,0.045158,0.016415
1784,2022-09-28,-0.011489,0.043311,0.015977
1785,2022-09-29,-0.013321,0.042562,0.015497
1786,2022-09-30,-0.012425,0.041572,0.015307
1787,2022-10-01,-0.014282,0.041742,0.014685


In [26]:
# Salvar modelo
with open('model_doge_usd.pckl', 'wb') as fout:
    pickle.dump(model, fout)